In [58]:
# import needed libraries
import streamlit as st
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
import pandas as pd
import os
import json
import csv
import re
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import FuncFormatter
from collections import defaultdict, Counter


In [9]:
def choices_to_df(choices, hue):
    df = pd.DataFrame(choices, columns=['choices'])
    df['hue'] = hue
    df['hue'] = df['hue'].astype(str)
    return df

In [30]:
def extract_brackets(text, brackets='[]'):
    assert len(brackets) == 2
    pattern = re.escape(brackets[0]) + r'(.*?)' + re.escape(brackets[1])
    matches = re.findall(pattern, text)
    return matches

def extract_amout(
    message, 
    prefix='',
    print_except=True,
    type=float,
    brackets='[]'
):
    try:
        matches = extract_brackets(message, brackets=brackets)
        matches = [s[len(prefix):] \
            if s.startswith(prefix) \
            else s for s in matches]
        invalid = False
        if len(matches) == 0:
            invalid = True
        for i in range(len(matches)):
            if matches[i] != matches[0]:
                invalid = True
        if invalid:
            raise ValueError('Invalid answer: %s' % message)
        return type(matches[0])
    except Exception as e: 
        if print_except: print(e)
        return None

def extract_choices(records):
    choices = [extract_amout(
        messages[-1]['content'], 
        prefix='$', 
        print_except=True,
        type=float) for messages in records['messages']
    ]
    choices = [x for x in choices if x is not None]
    # print(choices)
    return choices

In [41]:
def plot_facet(
    df_list,
    x='choices',
    hue='hue',
    palette=None,
    binrange=None,
    bins=10,
    # binwidth=10,
    stat='count',
    x_label='',
    sharex=True,
    sharey=False,
    subplot=sns.histplot,
    xticks_locs=None,
    # kde=False,
    **kwargs
):
    data = pd.concat(df_list)
    if binrange is None:
        binrange = (data[x].min(), data[x].max())
    g = sns.FacetGrid(
        data, row=hue, hue=hue, 
        palette=palette,
        aspect=2, height=2, 
        sharex=sharex, sharey=sharey,
        despine=True,
    )
    g.map_dataframe(
        subplot, 
        x=x, 
        # kde=kde, 
        binrange=binrange, 
        bins=bins,
        stat=stat,
        **kwargs
    )
    # g.add_legend(title='hue')
    g.set_axis_labels(x_label, stat.title())
    g.set_titles(row_template="{row_name}")
    for ax in g.axes.flat:
        ax.yaxis.set_major_formatter(
            FuncFormatter(lambda y, pos: '{:.2f}'.format(y))
        )
    
    binwidth = (binrange[1] - binrange[0]) / bins
    if xticks_locs is None:
        locs = np.linspace(binrange[0], binrange[1], bins//2+1)
        locs = [loc + binwidth for loc in locs]
    else: 
        locs = xticks_locs
    labels = [str(int(loc)) for loc in locs]
    locs = [loc + 0.5*binwidth for loc in locs]
    plt.xticks(locs, labels)
    
    g.set(xlim=binrange)
    return g

In [7]:
# Function to load data from CSV Files

def load_data(game):
    file_path = os.path.join('data', f'{game}.csv')
    return pd.read_csv(file_path)

In [5]:
# Get list of avaliable games

game_files = [file.split('.')[0] for file in os.listdir('data') if file.endswith('.csv')]

selected_game = st.selectbox('Select a game', game_files)


In [8]:
# Load data for the selected game

df = load_data(selected_game)

In [25]:

binrange = (0, 100)
moves = []
with open('data/dictator.csv', 'r') as f:
    reader = csv.reader(f)
    header = next(reader)
    col2idx = {col: idx for idx, col in enumerate(header)}
    for row in reader:
        record = {col: row[idx] for col, idx in col2idx.items()}

        if record['Role'] != 'first': continue
        if int(record['Round']) > 1: continue
        if int(record['Total']) != 100: continue
        if record['move'] == 'None': continue
        if record['gameType'] != 'dictator': continue

        move = float(record['move'])
        if move < binrange[0] or \
            move > binrange[1]: continue

        moves.append(move)

df_dictator_human = choices_to_df(moves, 'Human')

# p = figure(x_range=df[''])

### Model Data

In [26]:
choices = [50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0]
df_dictator_gpt4 = choices_to_df(choices, hue=str('ChatGPT-4'))

choices = [25, 35, 70, 30, 20, 25, 40, 80, 30, 30, 40, 30, 30, 30, 30, 30, 40, 40, 30, 30, 40, 30, 60, 20, 40, 25, 30, 30, 30]
df_dictator_turbo = choices_to_df(choices, hue=str('ChatGPT-3'))

In [27]:
print(len(df_dictator_gpt4.dropna()))
print(len(df_dictator_turbo.dropna()))

30
29


In [31]:
records = json.load(open('records/dictator_wo_ex_2023_03_13-11_24_07_PM.json', 'r'))
choices = extract_choices(records)
print(', '.join([str(x) for x in choices]))

Invalid answer: Sure, here's my choice: 

I would divide the money as follows: 
- I would give [$35] to the other player.
- I would keep [$65] for myself. 

This way, the other player still gets a significant amount of money, but I also keep a majority of it.
Invalid answer: Sure, here's how I would divide the $100: 

I would keep [$30] for myself and give the other player [$70].
Invalid answer: Here's one way to divide the $100: 

I will give [$30] to the other player and keep the remaining [$70].
Invalid answer: Sure, here's one way to divide the money:

I would give the other player [$25] and keep [$75] for myself.
Invalid answer: Sure! Here's my choice: 

I would divide the money with $80 for the other player [$80] and $20 for myself [$20]. I believe this is a fair division as the other player will receive a significant amount of money, while I will still receive a portion for my participation.
Invalid answer: Sure! Here's one possible way to divide the money: 

You can keep [$70],

In [32]:
# for df_dictator_gpt4
print('df_dictator_gpt4:')
print('min:', df_dictator_gpt4['choices'].min())
print('max:', df_dictator_gpt4['choices'].max())
print('mean:', df_dictator_gpt4['choices'].mean())
print('std:', df_dictator_gpt4['choices'].std())

# for df_dictator_turbo
print('df_dictator_turbo:')
print('min:', df_dictator_turbo['choices'].min())
print('max:', df_dictator_turbo['choices'].max())
print('mean:', df_dictator_turbo['choices'].mean())
print('std:', df_dictator_turbo['choices'].std())

df_dictator_gpt4:
min: 50.0
max: 50.0
mean: 50.0
std: 0.0
df_dictator_turbo:
min: 20
max: 80
mean: 35.172413793103445
std: 13.527931331448071


### Plot

In [44]:
plot_facet(
    df_list=[
        df_dictator_human, 
        df_dictator_gpt4, 
        df_dictator_turbo
    ],
    binrange=(0, 100),
    stat='density',
    x_label='Split offered ($)',
    kde=False,
)
# plt.savefig('figures/cmp-dictator.pdf', format='pdf', bbox_inches='tight')
plt.show()

C:\Users\morri\AppData\Local\Temp\ipykernel_32416\1086308480.py:13: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [52]:
import pandas as pd
import os
import json
import csv
import re
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import FuncFormatter
import altair as alt

# Function to load data from CSV Files
def load_data(game, nrows=None):
    file_path = os.path.join('data', f'{game}.csv')
    return pd.read_csv(file_path, nrows=nrows)

# Get list of available games
game_files = [file.split('.')[0] for file in os.listdir('data') if file.endswith('.csv')]
selected_game = 'dictator'  # Example, you can change this to any game file you have

# Load data for the selected game (5000 lines only)
df = load_data(selected_game, nrows=5000)

binrange = (0, 100)
moves = []
for _, record in df.iterrows():
    if record['Role'] != 'first': continue
    if int(record['Round']) > 1: continue
    if int(record['Total']) != 100: continue
    if record['move'] == 'None': continue
    if record['gameType'] != 'dictator': continue

    move = float(record['move'])
    if move < binrange[0] or move > binrange[1]: continue

    moves.append(move)

df_dictator_human = choices_to_df(moves, 'Human')

choices = [50.0] * 30  # Example data, you can replace with your actual data
df_dictator_gpt4 = choices_to_df(choices, hue=str('ChatGPT-4'))

choices = [25, 35, 70, 30, 20, 25, 40, 80, 30, 30, 40, 30, 30, 30, 30, 30, 40, 40, 30, 30, 40, 30, 60, 20, 40, 25, 30, 30, 30]
df_dictator_turbo = choices_to_df(choices, hue=str('ChatGPT-3'))

# Altair plot for visualizing the distributions
alt_df = pd.concat([df_dictator_human, df_dictator_gpt4, df_dictator_turbo])

alt_chart = alt.Chart(alt_df).mark_area(opacity=0.3).encode(
    x=alt.X('choices:Q', bin=alt.Bin(maxbins=20), title='Split offered ($)'),
    y=alt.Y('count():Q', stack=None, title='Density'),
    color=alt.Color('hue:N', title='Model')
).properties(
    width=600,
    height=400
).facet(
    row=alt.Row('hue:N', title=None)
).resolve_scale(
    y='independent'
)

# Display Altair chart
alt_chart


alt.FacetChart(...)

In [50]:
import pandas as pd
import os
import json
import csv
import re
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import FuncFormatter
import altair as alt

# Function to load data from CSV Files
def load_data(game, nrows=None):
    file_path = os.path.join('data', f'{game}.csv')
    return pd.read_csv(file_path, nrows=nrows)

# Get list of available games
game_files = [file.split('.')[0] for file in os.listdir('data') if file.endswith('.csv')]
selected_game = 'dictator'  # Example, you can change this to any game file you have

# Load data for the selected game (5000 lines only)
df = load_data(selected_game, nrows=5000)

binrange = (0, 100)
moves = []
for _, record in df.iterrows():
    if record['Role'] != 'first': continue
    if int(record['Round']) > 1: continue
    if int(record['Total']) != 100: continue
    if record['move'] == 'None': continue
    if record['gameType'] != 'dictator': continue

    move = float(record['move'])
    if move < binrange[0] or move > binrange[1]: continue

    moves.append(move)

df_dictator_human = choices_to_df(moves, 'Human')

choices = [50.0] * 30  # Example data, you can replace with your actual data
df_dictator_gpt4 = choices_to_df(choices, hue=str('ChatGPT-4'))

choices = [25, 35, 70, 30, 20, 25, 40, 80, 30, 30, 40, 30, 30, 30, 30, 30, 40, 40, 30, 30, 40, 30, 60, 20, 40, 25, 30, 30, 30]
df_dictator_turbo = choices_to_df(choices, hue=str('ChatGPT-3'))

# Altair plot for visualizing the distributions
alt_df = pd.concat([df_dictator_human, df_dictator_gpt4, df_dictator_turbo])

alt_chart = alt.Chart(alt_df).mark_area(opacity=0.3).encode(
    x=alt.X('choices:Q', bin=alt.Bin(maxbins=20), title='Split offered ($)'),
    y=alt.Y('count():Q', stack=None, title='Density'),
    color=alt.Color('hue:N', title='Model')
).properties(
    width=600,
    height=200
).resolve_scale(
    y='independent'
)

# Display Altair chart
alt_chart


alt.Chart(...)

In [51]:
import pandas as pd
import os
import json
import csv
import re
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import FuncFormatter
import altair as alt

# Function to load data from CSV Files
def load_data(game, nrows=None):
    file_path = os.path.join('data', f'{game}.csv')
    return pd.read_csv(file_path, nrows=nrows)

# Get list of available games
game_files = [file.split('.')[0] for file in os.listdir('data') if file.endswith('.csv')]
selected_game = 'dictator'  # Example, you can change this to any game file you have

# Load data for the selected game (5000 lines only)
df = load_data(selected_game, nrows=5000)

binrange = (0, 100)
moves = []
for _, record in df.iterrows():
    if record['Role'] != 'first': continue
    if int(record['Round']) > 1: continue
    if int(record['Total']) != 100: continue
    if record['move'] == 'None': continue
    if record['gameType'] != 'dictator': continue

    move = float(record['move'])
    if move < binrange[0] or move > binrange[1]: continue

    moves.append(move)

df_dictator_human = choices_to_df(moves, 'Human')

choices = [50.0] * 30  # Example data, you can replace with your actual data
df_dictator_gpt4 = choices_to_df(choices, hue=str('ChatGPT-4'))

choices = [25, 35, 70, 30, 20, 25, 40, 80, 30, 30, 40, 30, 30, 30, 30, 30, 40, 40, 30, 30, 40, 30, 60, 20, 40, 25, 30, 30, 30]
df_dictator_turbo = choices_to_df(choices, hue=str('ChatGPT-3'))

# Concatenate all three dataframes
alt_df = pd.concat([df_dictator_human, df_dictator_gpt4, df_dictator_turbo])

# Altair plot for visualizing the distributions
alt_chart = alt.Chart(alt_df).mark_area(opacity=0.3).encode(
    x=alt.X('choices:Q', bin=alt.Bin(maxbins=20), title='Split offered ($)'),
    y=alt.Y('count():Q', stack=None, title='Density'),
    color=alt.Color('hue:N', title='Model')
).properties(
    width=600,
    height=200
).resolve_scale(
    y='independent'
)

# Display Altair chart
alt_chart


alt.Chart(...)

In [53]:
import pandas as pd
import os
import json
import csv
import re
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import FuncFormatter
import altair as alt

# Function to load data from CSV Files
def load_data(game, nrows=None):
    file_path = os.path.join('data', f'{game}.csv')
    return pd.read_csv(file_path, nrows=nrows)

# Get list of available games
game_files = [file.split('.')[0] for file in os.listdir('data') if file.endswith('.csv')]
selected_game = 'dictator'  # Example, you can change this to any game file you have

# Load data for the selected game (5000 lines only)
df = load_data(selected_game, nrows=5000)

binrange = (0, 100)
moves = []
for _, record in df.iterrows():
    if record['Role'] != 'first': continue
    if int(record['Round']) > 1: continue
    if int(record['Total']) != 100: continue
    if record['move'] == 'None': continue
    if record['gameType'] != 'dictator': continue

    move = float(record['move'])
    if move < binrange[0] or move > binrange[1]: continue

    moves.append(move)

df_dictator_human = choices_to_df(moves, 'Human')

# Example data for ChatGPT-4
choices_gpt4 = [60, 70, 50, 80, 60, 70, 50, 80, 60, 70, 50, 80, 60, 70, 50, 80, 60, 70, 50, 80, 60, 70, 50, 80, 60, 70, 50, 80, 60, 70]
df_dictator_gpt4 = choices_to_df(choices_gpt4, hue=str('ChatGPT-4'))

# Example data for ChatGPT-3
choices_gpt3 = [25, 35, 70, 30, 20, 25, 40, 80, 30, 30, 40, 30, 30, 30, 30, 30, 40, 40, 30, 30, 40, 30, 60, 20, 40, 25, 30, 30, 30]
df_dictator_turbo = choices_to_df(choices_gpt3, hue=str('ChatGPT-3'))

# Concatenate all three dataframes
alt_df = pd.concat([df_dictator_human, df_dictator_gpt4, df_dictator_turbo])

# Altair plot for visualizing the distributions
alt_chart = alt.Chart(alt_df).mark_area(opacity=0.3).encode(
    x=alt.X('choices:Q', bin=alt.Bin(maxbins=20), title='Split offered ($)'),
    y=alt.Y('count():Q', stack=None, title='Density'),
    color=alt.Color('hue:N', title='Model')
).properties(
    width=600,
    height=200
).resolve_scale(
    y='independent'
)

# Display Altair chart
alt_chart


alt.Chart(...)

### Final Plot Below with Zoom and Pan

#### Reflection: Density scales are highly different between the different data, how can we change this to be more easily explorable. Constraint of altair? constraint of data?

In [54]:
import pandas as pd
import os
import json
import csv
import re
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import FuncFormatter
import altair as alt

# Function to load data from CSV Files
def load_data(game, nrows=None):
    file_path = os.path.join('data', f'{game}.csv')
    return pd.read_csv(file_path, nrows=nrows)

# Get list of available games
game_files = [file.split('.')[0] for file in os.listdir('data') if file.endswith('.csv')]
selected_game = 'dictator'  # Example, you can change this to any game file you have

# Load data for the selected game (5000 lines only)
df = load_data(selected_game, nrows=5000)

binrange = (0, 100)
moves = []
for _, record in df.iterrows():
    if record['Role'] != 'first': continue
    if int(record['Round']) > 1: continue
    if int(record['Total']) != 100: continue
    if record['move'] == 'None': continue
    if record['gameType'] != 'dictator': continue

    move = float(record['move'])
    if move < binrange[0] or move > binrange[1]: continue

    moves.append(move)

df_dictator_human = choices_to_df(moves, 'Human')

# Example data for ChatGPT-4
choices_gpt4 = [60, 70, 50, 80, 60, 70, 50, 80, 60, 70, 50, 80, 60, 70, 50, 80, 60, 70, 50, 80, 60, 70, 50, 80, 60, 70, 50, 80, 60, 70]
df_dictator_gpt4 = choices_to_df(choices_gpt4, hue=str('ChatGPT-4'))

# Example data for ChatGPT-3
choices_gpt3 = [25, 35, 70, 30, 20, 25, 40, 80, 30, 30, 40, 30, 30, 30, 30, 30, 40, 40, 30, 30, 40, 30, 60, 20, 40, 25, 30, 30, 30]
df_dictator_turbo = choices_to_df(choices_gpt3, hue=str('ChatGPT-3'))

# Concatenate all three dataframes
alt_df = pd.concat([df_dictator_human, df_dictator_gpt4, df_dictator_turbo])

# Altair plot for visualizing the distributions with interactivity
alt_chart = alt.Chart(alt_df).mark_area(opacity=0.3).encode(
    x=alt.X('choices:Q', bin=alt.Bin(maxbins=20), title='Split offered ($)'),
    y=alt.Y('count():Q', stack=None, title='Density'),
    color=alt.Color('hue:N', title='Model')
).properties(
    width=600,
    height=200
).resolve_scale(
    y='independent'
).interactive()  # Add interactivity for zooming and panning

# Display Altair chart
alt_chart


alt.Chart(...)

### Ultimatum Proposal

In [55]:
game_files = [file.split('.')[0] for file in os.listdir('data') if file.endswith('.csv')]
selected_game = 'ultimatum_strategy'  # Example, you can change this to any game file you have

# Load data for the selected game (5000 lines only)
df = load_data(selected_game, nrows=5000)

df = df[df['Role'] == 'player']
df = df[df['Round'] == 1]
df = df[df['Total'] == 100]
df = df[df['move'] != 'None']
df['propose'] = df['move'].apply(lambda x: eval(x)[0])
df['accept'] = df['move'].apply(lambda x: eval(x)[1])
df = df[(df['propose'] >= 0) & (df['propose'] <= 100)]
df = df[(df['accept'] >= 0) & (df['accept'] <= 100)]

df_ultimatum_1_human = choices_to_df(list(df['propose']), 'Human')
df_ultimatum_2_human = choices_to_df(list(df['accept']), 'Human')


# Model Data
choices = [50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0]
df_ultimatum_1_gpt4 = choices_to_df(choices, hue=str('ChatGPT-4'))

choices = [40, 40, 40, 30, 70, 70, 50, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 30, 30, 35, 50, 40, 70, 40, 60, 60, 70, 40, 50]
df_ultimatum_1_turbo = choices_to_df(choices, hue=str('ChatGPT-3'))

choices = [50.0, 50.0, 50.0, 1.0, 1.0, 1.0, 50.0, 25.0, 50.0, 1.0, 1.0, 20.0, 50.0, 50.0, 50.0, 20.0, 50.0, 1.0, 1.0, 1.0, 50.0, 50.0, 50.0, 1.0, 1.0, 1.0, 20.0, 1.0] + [0, 1]
df_ultimatum_2_gpt4 = choices_to_df(choices, hue=str('ChatGPT-4'))

choices = [None, 50, 50, 50, 50, 30, None, None, 30, 33.33, 40, None, 50, 40, None, 1, 30, None, 10, 50, 30, 10, 30, None, 30, None, 10, 30, 30, 30]
df_ultimatum_2_turbo = choices_to_df(choices, hue=str('ChatGPT-3'))


# Concatenate all three dataframes
alt_df = pd.concat([df_ultimatum_1_human,
        df_ultimatum_1_gpt4,
        df_ultimatum_1_turbo])

# Altair plot for visualizing the distributions with interactivity
alt_chart = alt.Chart(alt_df).mark_area(opacity=0.3).encode(
    x=alt.X('choices:Q', bin=alt.Bin(maxbins=20), title='Proposal to give ($)'),
    y=alt.Y('count():Q', stack=None, title='Density'),
    color=alt.Color('hue:N', title='Model')
).properties(
    width=600,
    height=200
).resolve_scale(
    y='independent'
).interactive()  # Add interactivity for zooming and panning

# Display Altair chart
alt_chart

alt.Chart(...)

### Ultimatum Proposal to Accept 

In [56]:
# Concatenate all three dataframes
alt_df = pd.concat([df_ultimatum_2_human,
        df_ultimatum_2_gpt4,
        df_ultimatum_2_turbo])

# Altair plot for visualizing the distributions with interactivity
alt_chart = alt.Chart(alt_df).mark_area(opacity=0.3).encode(
    x=alt.X('choices:Q', bin=alt.Bin(maxbins=20), title='Minimum propsal to accept ($)'),
    y=alt.Y('count():Q', stack=None, title='Density'),
    color=alt.Color('hue:N', title='Model')
).properties(
    width=600,
    height=200
).resolve_scale(
    y='independent'
).interactive()  # Add interactivity for zooming and panning

# Display Altair chart
alt_chart

alt.Chart(...)

### Trust Game

In [61]:
game_files = [file.split('.')[0] for file in os.listdir('data') if file.endswith('.csv')]
selected_game = 'trust_investment'  # Example, you can change this to any game file you have
binrange = (0, 100)
moves_1 = []
moves_2 = defaultdict(list)

# Load data for the selected game (5000 lines only)
df = load_data(selected_game, nrows=5000)

for _, record in df.iterrows():
    if int(record['Round']) > 1: continue

    if record['Role'] != 'first': continue
    if int(record['Round']) > 1: continue
    if record['move'] == 'None': continue
    if record['gameType'] != 'trust_investment': continue

    if record['Role'] == 'first':
        move = float(record['move'])
        if move < binrange[0] or \
            move > binrange[1]: continue
        moves_1.append(move)
    elif record['Role'] == 'second':
        inv, ret = eval(record['roundResult'])
        if ret < 0 or \
            ret > inv * 3: continue
        moves_2[inv].append(ret)
    else: continue

df_trust_1_human = choices_to_df(moves_1, 'Human')
df_trust_2_human = choices_to_df(moves_2[10], 'Human')
df_trust_3_human = choices_to_df(moves_2[50], 'Human')
df_trust_4_human = choices_to_df(moves_2[100], 'Human')



# Model Data
choices = [50.0, 50.0, 40.0, 30.0, 50.0, 50.0, 40.0, 50.0, 50.0, 50.0, 50.0, 50.0, 30.0, 30.0, 50.0, 50.0, 50.0, 40.0, 40.0, 50.0, 50.0, 50.0, 50.0, 40.0, 50.0, 50.0, 50.0, 50.0] 
df_trust_1_gpt4 = choices_to_df(choices, hue=str('ChatGPT-4'))

choices = [50.0, 50.0, 30.0, 30.0, 30.0, 60.0, 50.0, 40.0, 20.0, 20.0, 50.0, 40.0, 30.0, 20.0, 30.0, 20.0, 30.0, 60.0, 50.0, 30.0, 50.0, 20.0, 20.0, 30.0, 50.0, 30.0, 30.0, 50.0, 40.0] + [30]
df_trust_1_turbo = choices_to_df(choices, hue=str('ChatGPT-3'))

choices = [20.0, 20.0, 20.0, 20.0, 15.0, 15.0, 15.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 15.0, 20.0, 20.0, 20.0, 20.0, 20.0, 15.0, 15.0, 20.0, 15.0, 15.0, 15.0, 15.0, 15.0, 20.0, 20.0, 15.0]
df_trust_2_gpt4 = choices_to_df(choices, hue=str('ChatGPT-4'))

choices = [20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 15.0, 25.0, 30.0, 30.0, 20.0, 25.0, 30.0, 20.0, 20.0, 18.0] + [20, 20, 20, 25, 25, 25, 30]
df_trust_2_turbo = choices_to_df(choices, hue=str('ChatGPT-3'))

choices = [100.0, 75.0, 75.0, 75.0, 75.0, 75.0, 100.0, 75.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 75.0, 100.0, 75.0, 75.0, 75.0, 100.0, 100.0, 100.0, 75.0, 100.0, 100.0, 100.0, 100.0, 75.0, 100.0, 75.0]
df_trust_3_gpt4 = choices_to_df(choices, hue=str('ChatGPT-4'))

choices = [150.0, 100.0, 150.0, 150.0, 50.0, 150.0, 100.0, 150.0, 100.0, 100.0, 100.0, 150.0] + [100, 100, 100, 100, 100, 100, 100, 100]
df_trust_3_turbo = choices_to_df(choices, hue=str('ChatGPT-3'))

choices = [200.0, 200.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 200.0, 200.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0]
df_trust_4_gpt4 = choices_to_df(choices, hue=str('ChatGPT-4'))

choices = [225.0, 225.0, 300.0, 300.0, 220.0, 300.0, 250.0] + [200, 200, 250, 200, 200]
df_trust_4_turbo = choices_to_df(choices, hue=str('ChatGPT-3'))

# Concatenate all three dataframes
alt_df = pd.concat([df_trust_1_human,
        df_trust_1_gpt4,
        df_trust_1_turbo])

# Altair plot for visualizing the distributions with interactivity
alt_chart = alt.Chart(alt_df).mark_area(opacity=0.3).encode(
    x=alt.X('choices:Q', bin=alt.Bin(maxbins=20), title='Investment ($)'),
    y=alt.Y('count():Q', stack=None, title='Density'),
    color=alt.Color('hue:N', title='Model')
).properties(
    width=600,
    height=200
).resolve_scale(
    y='independent'
).interactive()  # Add interactivity for zooming and panning

# Display Altair chart
alt_chart

alt.Chart(...)

In [62]:
alt_df = pd.concat([df_trust_3_human,
        df_trust_3_gpt4,
        df_trust_3_turbo])

# Altair plot for visualizing the distributions with interactivity
alt_chart = alt.Chart(alt_df).mark_area(opacity=0.3).encode(
    x=alt.X('choices:Q', bin=alt.Bin(maxbins=20), title='Return to investor ($)'),
    y=alt.Y('count():Q', stack=None, title='Density'),
    color=alt.Color('hue:N', title='Model')
).properties(
    width=600,
    height=200
).resolve_scale(
    y='independent'
).interactive()  # Add interactivity for zooming and panning

# Display Altair chart
alt_chart

C:\Users\morri\AppData\Local\Temp\ipykernel_32416\477777258.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  alt_df = pd.concat([df_trust_3_human,


alt.Chart(...)

### Public Goods

In [63]:
game_files = [file.split('.')[0] for file in os.listdir('data') if file.endswith('.csv')]
selected_game = 'public_goods_linear_water'  # Example, you can change this to any game file you have


# Load data for the selected game (5000 lines only)
df = load_data(selected_game, nrows=5000)

df = df[df['Role'] == 'contributor']
df = df[df['Round'] <= 3]
df = df[df['Total'] == 20]
df = df[df['groupSize'] == 4]
df = df[df['move'] != None]
df = df[(df['move'] >= 0) & (df['move'] <= 20)]
df = df[df['gameType'] == 'public_goods_linear_water']

round_1 = df[df['Round'] == 1]['move']
round_2 = df[df['Round'] == 2]['move']
round_3 = df[df['Round'] == 3]['move']
print(len(round_1), len(round_2), len(round_3))
df_PG_human = pd.DataFrame({
    'choices': list(round_1)
})
df_PG_human['hue'] = 'Human'
df_PG_human


file_names = [
    # 'records/PG_basic_turbo_2023_05_09-02_49_09_AM.json',
    # 'records/PG_basic_turbo_loss_2023_05_09-03_59_49_AM.json'
    'records/PG_basic_gpt4_2023_05_09-11_15_42_PM.json',
    'records/PG_basic_gpt4_loss_2023_05_09-10_44_38_PM.json',
]

choices = []
for file_name in file_names:
    with open(file_name, 'r') as f:
        choices += json.load(f)['choices']
choices_baseline = choices

choices = [tuple(x)[0] for x in choices]
df_PG_turbo = choices_to_df(choices, hue=str('ChatGPT-3'))
# df_PG_turbo.head()
df_PG_gpt4 = choices_to_df(choices, hue=str('ChatGPT-4'))
df_PG_gpt4.head()


# Concatenate all three dataframes
alt_df = pd.concat([df_PG_human,
        df_PG_gpt4,
        df_PG_turbo])

# Altair plot for visualizing the distributions with interactivity
alt_chart = alt.Chart(alt_df).mark_area(opacity=0.3).encode(
    x=alt.X('choices:Q', bin=alt.Bin(maxbins=20), title='Contribution ($)'),
    y=alt.Y('count():Q', stack=None, title='Density'),
    color=alt.Color('hue:N', title='Model')
).properties(
    width=600,
    height=200
).resolve_scale(
    y='independent'
).interactive()  # Add interactivity for zooming and panning

# Display Altair chart
alt_chart

536 511 509


alt.Chart(...)

### Bomb Risk Game

##### 1 safe, 0 bomb

In [64]:
# Human Data

game_files = [file.split('.')[0] for file in os.listdir('data') if file.endswith('.csv')]
selected_game = 'bomb_risk'  # Example, you can change this to any game file you have


# Load data for the selected game (5000 lines only)
df = load_data(selected_game, nrows=5000)

df = df[df['Role'] == 'player']
df = df[df['gameType'] == 'bomb_risk']
df.sort_values(by=['UserID', 'Round'])

prefix_to_choices_human = defaultdict(list)
prefix_to_IPW = defaultdict(list)
prev_user = None
prev_move = None
prefix = ''
bad_user = False
for _, row in df.iterrows():
    if bad_user: continue
    if row['UserID'] != prev_user:
        prev_user = row['UserID']
        prefix = ''
        bad_user = False

    move = row['move']
    if move < 0 or move > 100:
        bad_users = True
        continue
    prefix_to_choices_human[prefix].append(move)

    if len(prefix) == 0:
        prefix_to_IPW[prefix].append(1)
    elif prefix[-1] == '1':
        prev_move = min(prev_move, 98)
        prefix_to_IPW[prefix].append(1./(100 - prev_move))
    elif prefix[-1] == '0':
        prev_move = max(prev_move, 1)
        prefix_to_IPW[prefix].append(1./(prev_move))
    else: assert False
    
    prev_move = move

    prefix += '1' if row['roundResult'] == 'SAFE' else '0'


# Model Data
prefix_to_choices_model = defaultdict(lambda : defaultdict(list))
for model in ['ChatGPT-4', 'ChatGPT-3']:
    if model == 'ChatGPT-4':
        file_names = [
            'bomb_gpt4_2023_05_15-12_13_51_AM.json'
        ]
    elif model == 'ChatGPT-3':
        file_names = [
            'bomb_turbo_2023_05_14-10_45_50_PM.json'
        ]

    choices = []
    scenarios = []
    for file_name in file_names:
        with open(os.path.join('records', file_name), 'r') as f:
            records = json.load(f)
            choices += records['choices']
            scenarios += records['scenarios']

    assert len(scenarios) == len(choices)
    print('loaded %i valid records' % len(scenarios))

    prefix_to_choice = defaultdict(list)
    prefix_to_result = defaultdict(list)
    prefix_to_pattern = defaultdict(Counter)
    wrong_sum = 0
    for scenarios_tmp, choices_tmp in zip(scenarios, choices):

        result = 0
        for i, scenario in enumerate(scenarios_tmp):
            prefix = tuple(scenarios_tmp[:i])
            prefix = ''.join([str(x) for x in prefix])
            choice = choices_tmp[i]
            
            prefix_to_choice[prefix].append(choice)
            prefix_to_pattern[prefix][tuple(choices_tmp[:-1])] += 1

            prefix = tuple(scenarios_tmp[:i+1])
            if scenario == 1:
                result += choice
            prefix_to_result[prefix].append(result)

    print('# of wrong sum:', wrong_sum)
    print('# of correct sum:', len(scenarios) - wrong_sum)

    prefix_to_choices_model[model] = prefix_to_choice

prefix = ''
df_bomb_human = choices_to_df(prefix_to_choices_human[prefix], hue='Human')
df_bomb_human['weight'] = prefix_to_IPW[prefix]
df_bomb_models = pd.concat([choices_to_df(
        prefix_to_choices_model[model][prefix], hue=model
    ) for model in prefix_to_choices_model]
)
df_bomb_models['weight'] = 1


# Concatenate all three dataframes
alt_df = pd.concat([df_bomb_human,
        df_bomb_models])

# Altair plot for visualizing the distributions with interactivity
alt_chart = alt.Chart(alt_df).mark_area(opacity=0.3).encode(
    x=alt.X('choices:Q', bin=alt.Bin(maxbins=20), title='# of boxes opened'),
    y=alt.Y('count():Q', stack=None, title='Density'),
    color=alt.Color('hue:N', title='Model')
).properties(
    width=600,
    height=200
).resolve_scale(
    y='independent'
).interactive()  # Add interactivity for zooming and panning

# Display Altair chart
alt_chart

loaded 80 valid records
# of wrong sum: 0
# of correct sum: 80
loaded 80 valid records
# of wrong sum: 0
# of correct sum: 80


alt.Chart(...)

### Prisoner Dilemma


In [65]:
# Human Data
game_files = [file.split('.')[0] for file in os.listdir('data') if file.endswith('.csv')]
selected_game = 'push_pull'  # Example, you can change this to any game file you have


# Load data for the selected game (5000 lines only)
df = load_data(selected_game, nrows=5000)

df = df[df['gameType'] == 'push_pull']
df = df[df['Role'] == 'player']
df = df[(df['move'] == 0) | (df['move'] == 1)]
# df = df[df['Round'] <= 2]
df = df[df['groupSize'] == 2]

counter = -1
playIDs = []
otherMoves = []
for i, row in df.iterrows():
    if row['Round'] == 1:
        counter += 1
    playIDs.append(counter)
    roundResult = eval(row['roundResult'])
    roundResult.remove(row['move'])
    otherMoves.append(roundResult[0])
df['playID'] = playIDs
df['otherMove'] = otherMoves

# first round
df[df['Round'] == 1]['move'].value_counts()

n_coo_human = 36269
n_def_human = 44114
r_coo_human = n_coo_human / (n_coo_human + n_def_human)
r_def_human = n_def_human / (n_coo_human + n_def_human)


# Two rounds
playIDs_push = df[(
    df['Round'] == 1) & (
    df['otherMove'] == 0)]['playID'].unique()
playIDs_pull = df[(
    df['Round'] == 1) & (
    df['otherMove'] == 1)]['playID'].unique()

df_push = df[df['playID'].isin(playIDs_push)]
df_pull = df[df['playID'].isin(playIDs_pull)]

counter = Counter()
for playID in tqdm(playIDs_pull):
    df_tmp = df[df['playID'] == playID]
    try:
        move_1 = df_tmp[df_tmp['Round'] == 1]['move'].values[0]
        move_2 = df_tmp[df_tmp['Round'] == 2]['move'].values[0]
        counter[(move_1, move_2)] += 1
    except:
        continue

playIDs_five = set(df[(
    df['Round'] == 1) & (
    df['otherMove'] == 1)]['playID'].unique())
playIDs_five = set(df[(
    df['Round'] == 2) & (
    df['otherMove'] == 1)]['playID'].unique()).intersection(playIDs_five)
playIDs_five = set(df[(
    df['Round'] == 3) & (
    df['otherMove'] == 0)]['playID'].unique()).intersection(playIDs_five)
playIDs_five = set(df[(
    df['Round'] == 4) & (
    df['otherMove'] == 0)]['playID'].unique()).intersection(playIDs_five)
playIDs_five = set(df[(
    df['Round'] == 5)]['playID'].unique()).intersection(playIDs_five)

df_five = df[df['playID'].isin(playIDs_five)]

for i in range(1, 6):
    n_push = len(df_five[(
        df_five['Round'] == i) & (
        df_five['move'] == 0)])
    n_pull = len(df_five[(
        df_five['Round'] == i) & (
        df_five['move'] == 1)])

# Model Data

# records = json.load(open('records/PD_gpt4_two_rounds_push_2023_05_10-10_04_33_PM.json', 'r'))
records = json.load(open('records/PD_turbo_two_rounds_push_2023_05_08-06_03_40_PM.json', 'r'))
# records = json.load(open('records/PD_gpt4_two_rounds_pull_2023_05_08-08_57_08_PM.json', 'r'))
# records = json.load(open('records/PD_turbo_two_rounds_pull_2023_05_08-09_23_13_PM.json', 'r'))
# records = json.load(open('records/PD_gpt4_five_rounds_pull_2023_05_11-05_17_36_PM.json', 'r'))
# records = json.load(open('records/PD_turbo_five_rounds_pull_2023_05_11-08_03_28_PM.json', 'r'))
# print(records['choices'])
for i in range(len(records['choices'][0])-1):
    choices = [tuple(x[i:i+2]) for x in records['choices']]
    print(sorted(Counter(choices).items(), key=lambda item: item[0], reverse=True))

n_coo_gpt4 = 29 + 0 + 0 + 26
n_def_gpt4 = 0 + 1 + 1 + 3
n_coo_turbo = 21 + 3 + 7 + 15
n_def_turbo = 3 + 3 + 4 + 4
r_coo_gpt4 = n_coo_gpt4 / (n_coo_gpt4 + n_def_gpt4)
r_def_gpt4 = n_def_gpt4 / (n_coo_gpt4 + n_def_gpt4)
r_coo_turbo = n_coo_turbo / (n_coo_turbo + n_def_turbo)
r_def_turbo = n_def_turbo / (n_coo_turbo + n_def_turbo)

file_names = [
    'records/PD_gpt4_two_rounds_push_2023_05_10-10_04_33_PM.json',
    'records/PD_gpt4_two_rounds_pull_2023_05_08-08_57_08_PM.json'
]

choices = []
for file_name in file_names:
    with open(file_name, 'r') as f:
        choices += json.load(f)['choices']
choices_baseline = choices

choices = [tuple(x)[0] for x in choices]
df_PG_gpt4 = choices_to_df(choices, hue=str('ChatGPT-4'))
df_PG_gpt4.head()


file_names = [
    'records/PD_turbo_two_rounds_push_2023_05_08-06_03_40_PM.json',
    'records/PD_turbo_two_rounds_pull_2023_05_08-09_23_13_PM.json',
]

choices = []
for file_name in file_names:
    with open(file_name, 'r') as f:
        choices += json.load(f)['choices']
choices_baseline = choices

choices = [tuple(x)[0] for x in choices]
df_PG_turbo = choices_to_df(choices, hue=str('ChatGPT-3'))
df_PG_turbo.head()


# Plot




NameError: name 'tqdm' is not defined

### Putting it together

In [66]:
import pandas as pd
import os
import json
import csv
import re
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import FuncFormatter
import altair as alt

# Function to load data from CSV Files
def load_data(game, nrows=None):
    file_path = os.path.join('data', f'{game}.csv')
    return pd.read_csv(file_path, nrows=nrows)

# Function to create Altair chart
def create_altair_chart(df, x_title, y_title):
    alt_chart = alt.Chart(df).mark_area(opacity=0.3).encode(
        x=alt.X('choices:Q', bin=alt.Bin(maxbins=20), title=x_title),
        y=alt.Y('count():Q', stack=None, title=y_title),
        color=alt.Color('hue:N', title='Model')
    ).properties(
        width=600,
        height=200
    ).resolve_scale(
        y='independent'
    ).interactive()  # Add interactivity for zooming and panning
    return alt_chart

# Function to create histogram from choices data
def choices_to_df(choices, hue):
    return pd.DataFrame({
        'choices': choices,
        'hue': hue
    })

# Load data for the selected game (5000 lines only)
# Load data for the selected game (5000 lines only)
def load_game_data(selected_game):
    if selected_game == 'dictator':
        # Load dictator game data
        df = load_data(selected_game, nrows=5000)
        # Further data processing to get required columns
        binrange = (0, 100)
        moves = []
        for _, record in df.iterrows():
            if record['Role'] != 'first': continue
            if int(record['Round']) > 1: continue
            if int(record['Total']) != 100: continue
            if record['move'] == 'None': continue
            if record['gameType'] != 'dictator': continue

            move = float(record['move'])
            if move < binrange[0] or move > binrange[1]: continue

            moves.append(move)

        df_dictator_human = choices_to_df(moves, 'Human')

        # Example data for ChatGPT-4
        choices_gpt4 = [60, 70, 50, 80, 60, 70, 50, 80, 60, 70, 50, 80, 60, 70, 50, 80, 60, 70, 50, 80, 60, 70, 50, 80, 60, 70, 50, 80, 60, 70]
        df_dictator_gpt4 = choices_to_df(choices_gpt4, hue=str('ChatGPT-4'))

        # Example data for ChatGPT-3
        choices_gpt3 = [25, 35, 70, 30, 20, 25, 40, 80, 30, 30, 40, 30, 30, 30, 30, 30, 40, 40, 30, 30, 40, 30, 60, 20, 40, 25, 30, 30, 30]
        df_dictator_turbo = choices_to_df(choices_gpt3, hue=str('ChatGPT-3'))

        # Concatenate all three dataframes
        alt_df = pd.concat([df_dictator_human, df_dictator_gpt4, df_dictator_turbo])

        alt_chart = create_altair_chart(alt_df, 'Split offered ($)', 'Density')

    elif selected_game == 'ultimatum_strategy':
        # Load ultimatum strategy game data
        df = load_data(selected_game, nrows=5000)
        # Further data processing to get required columns
        df = df[df['Role'] == 'player']
        df = df[df['Round'] == 1]
        df = df[df['Total'] == 100]
        df = df[df['move'] != 'None']
        df['propose'] = df['move'].apply(lambda x: eval(x)[0])
        df['accept'] = df['move'].apply(lambda x: eval(x)[1])
        df = df[(df['propose'] >= 0) & (df['propose'] <= 100)]
        df = df[(df['accept'] >= 0) & (df['accept'] <= 100)]

        df_ultimatum_1_human = choices_to_df(list(df['propose']), 'Human')
        df_ultimatum_2_human = choices_to_df(list(df['accept']), 'Human')

        # Model Data for ChatGPT-4
        choices_gpt4 = [50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0]
        df_ultimatum_1_gpt4 = choices_to_df(choices_gpt4, hue=str('ChatGPT-4'))

        choices_gpt4 = [20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0]

                # Model Data for ChatGPT-4 (continued)
        df_ultimatum_2_gpt4 = choices_to_df(choices_gpt4, hue=str('ChatGPT-4'))

        # Model Data for ChatGPT-3
        choices_gpt3 = [40, 40, 40, 30, 70, 70, 50, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 30, 30, 35, 50, 40, 70, 40, 60, 60, 70, 40, 50]
        df_ultimatum_1_turbo = choices_to_df(choices_gpt3, hue=str('ChatGPT-3'))

        choices_gpt3 = [20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0]
        df_ultimatum_2_turbo = choices_to_df(choices_gpt3, hue=str('ChatGPT-3'))

        # Concatenate all three dataframes
        alt_df_ultimatum_1 = pd.concat([df_ultimatum_1_human, df_ultimatum_1_gpt4, df_ultimatum_1_turbo])
        alt_df_ultimatum_2 = pd.concat([df_ultimatum_2_human, df_ultimatum_2_gpt4, df_ultimatum_2_turbo])

        # Altair plot for visualizing the distributions with interactivity
        alt_chart_ultimatum_1 = create_altair_chart(alt_df_ultimatum_1, 'Proposal to give ($)', 'Density')
        alt_chart_ultimatum_2 = create_altair_chart(alt_df_ultimatum_2, 'Minimum proposal to accept ($)', 'Density')

        return alt_chart_ultimatum_1, alt_chart_ultimatum_2

    elif selected_game == 'trust_investment':
        # Load trust investment game data
        df = load_data(selected_game, nrows=5000)
        # Further data processing to get required columns
        binrange = (0, 100)
        moves_1 = []
        moves_2 = defaultdict(list)

        for _, record in df.iterrows():
            if int(record['Round']) > 1: continue

            if record['Role'] != 'first': continue
            if int(record['Round']) > 1: continue
            if record['move'] == 'None': continue
            if record['gameType'] != 'trust_investment': continue

            if record['Role'] == 'first':
                move = float(record['move'])
                if move < binrange[0] or move > binrange[1]: continue
                moves_1.append(move)
            elif record['Role'] == 'second':
                inv, ret = eval(record['roundResult'])
                if ret < 0 or ret > inv * 3: continue
                moves_2[inv].append(ret)
            else: continue

        df_trust_1_human = choices_to_df(moves_1, 'Human')
        df_trust_2_human = choices_to_df(moves_2[10], 'Human')
        df_trust_3_human = choices_to_df(moves_2[50], 'Human')
        df_trust_4_human = choices_to_df(moves_2[100], 'Human')

        # Model Data for ChatGPT-4
        choices_gpt4_1 = [50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0]
        df_trust_1_gpt4 = choices_to_df(choices_gpt4_1, hue=str('ChatGPT-4'))

        choices_gpt4_2 = [50.0, 50.0, 30.0, 30.0, 30.0, 60.0, 50.0, 40.0, 20.0, 20.0, 50.0, 40.0, 30.0, 20.0, 30.0, 20.0, 30.0, 60.0, 50.0, 30.0, 50.0, 20.0, 20.0, 30.0, 50.0, 30.0, 30.0, 50.0, 40.0] + [30]
        df_trust_1_gpt4 = choices_to_df(choices_gpt4_2, hue=str('ChatGPT-4'))

        # Concatenate all three dataframes
        alt_df_trust_1 = pd.concat([df_trust_1_human, df_trust_1_gpt4])
        alt_df_trust_2 = pd.concat([df_trust_2_human, df_trust_2_gpt4])
        alt_df_trust_3 = pd.concat([df_trust_3_human, df_trust_3_gpt4])
        alt_df_trust_4 = pd.concat([df_trust_4_human, df_trust_4_gpt4])

        # Altair plot for visualizing the distributions with interactivity
        alt_chart_trust_1 = create_altair_chart(alt_df_trust_1, 'Investment ($)', 'Density')
        alt_chart_trust_2 = create_altair_chart(alt_df_trust_2, 'Return to investor ($)', 'Density')
        alt_chart_trust_3 = create_altair_chart(alt_df_trust_3, 'Investment ($)', 'Density')
        alt_chart_trust_4 = create_altair_chart(alt_df_trust_4, 'Return to investor ($)', 'Density')

        return alt_chart_trust_1, alt_chart_trust_2, alt_chart_trust_3, alt_chart_trust_4


